<a href="https://colab.research.google.com/github/44REAM/radianet/blob/master/final_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:

!pip install optuna
!pip install pytorch_lightning
!pip install efficientnet_pytorch

%cd '/content/drive/My Drive/deeplearning'
!rm -r radianet
!git clone https://github.com/44REAM/radianet.git
%cd radianet


     |████████████████████████████████| 184kB 4.6MB/s 
     |████████████████████████████████| 1.1MB 14.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 112kB 25.1MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=1b9a93425e139b52acbb33f5fa0775bb92cb6cfb29a4853f28479a203c77ea06
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-1.4.0-cp36-none-any.whl size=254554 sha256=101ca86d76fc00ee893d061ca5a5cd8593c0f367022dc671c79dc33fdcb193d9
  Stored in directory: /root/.cache/pip/wheels/3e/

In [62]:
%load_ext tensorboard

import sys
import os
sys.path.append('..')

import torch
import torch.nn as nn
import optuna
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping

from radianet import get_dataloader
from radianet.datasets import SampleDataset2D, LIDCDataset, Transforms
from radianet.models import Simple3DCNN, MyEfficientNet
from radianet.callbacks import MetricsCallback
from radianet.lightningnet import LightningNet


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
class Config():
    BATCHSIZE = 32
    N_TRIALS = 1
    EPOCHS = 50
    N_CLASSES = 1
    BORDER_MODE = 4
    NAME = 'homework'
    IMAGE_SIZE = 224
    MIN_LR = 1e-6
    MAX_LR = 1e-4
    LIDC_PATH = '/content/drive/My Drive/dataset/radiology/TCIA_LIDC-IDRI/preprocess/'
    EFFICIENTNET_B0_LAYER = 0

    CHECKPOINT_DIR = '/content/drive/My Drive/deeplearning/checkpoints/'
    CHECKPOINT = CHECKPOINT_DIR + NAME
    TENSORBOARD = CHECKPOINT_DIR + NAME
    DB_NAME = 'sqlite:///' + NAME + '.db'


In [0]:
def get_logger():
    return TensorBoardLogger(Config.TENSORBOARD)

def get_model_checkpoint(trial):
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        os.path.join(Config.CHECKPOINT, "trial_{}".format(trial.number), "{epoch}"), 
        monitor="val_loss", save_top_k = -1
    )
    return checkpoint_callback



In [101]:

def objective(trial):

    logger = get_logger()
    checkpoint_callback = get_model_checkpoint(trial)
    metrics_callback = MetricsCallback()

    loss = nn.BCEWithLogitsLoss(pos_weight  = torch.tensor([0.307]))
    early_stop_callback = EarlyStopping(
        monitor='val_loss',
        min_delta=0.00,
        patience=3,
        verbose=False,
        mode='max'
    )
    # callbacks DO NOT replace the explicit callbacks (loggers, EarlyStopping or ModelCheckpoint)
    trainer = pl.Trainer(
        max_epochs=Config.EPOCHS,
        
        # all callbacks
        logger=logger,
        checkpoint_callback=checkpoint_callback,
        early_stop_callback = None,
        callbacks=[metrics_callback],

        gpus=[0] if torch.cuda.is_available() else None
    )
    
    transforms = Transforms(elastic_transform = False)
    datasets = LIDCDataset(transforms, Config.LIDC_PATH)
    #datasets = SampleDataset2D(transforms, n_sample = 10)
    dataloader = get_dataloader(datasets, Config.BATCHSIZE)

    lightning_model = LightningNet(trial, Config, MyEfficientNet, dataloader, loss)

    trainer.fit(lightning_model)
    trainer.test()

    return metrics_callback.metrics[-1]["val_loss"]

study = optuna.create_study(study_name = Config.NAME, storage = Config.DB_NAME, 
                            load_if_exists = True, direction="maximize")
study.optimize(objective, n_trials= Config.N_TRIALS)


[I 2020-05-13 08:41:09,928] Using an existing study with name 'homework' instead of creating a new one.
INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]


Loaded pretrained weights for efficientnet-b0


INFO:lightning:
    | Name                                              | Type                    | Params
------------------------------------------------------------------------------------------
0   | model                                             | MyEfficientNet          | 5 M   
1   | model.head                                        | Sequential              | 928   
2   | model.head.0                                      | Conv2dStaticSamePadding | 864   
3   | model.head.0.static_padding                       | ZeroPad2d               | 0     
4   | model.head.1                                      | BatchNorm2d             | 64    
5   | model.body_no_grad                                | Sequential              | 0     
6   | model.body_grad                                   | Sequential              | 3 M   
7   | model.body_grad.0                                 | MBConvBlock             | 1 K   
8   | model.body_grad.0._depthwise_conv                 | Conv2dStaticSame

linear input not have been calculate


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning:

The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.



/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning:

The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.



/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning:

The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.



--------------------------------------------------------------------------------
TEST RESULTS
{'test_acc': tensor(72.9114)}
--------------------------------------------------------------------------------



[I 2020-05-13 08:45:12,695] Finished trial#21 with value: 74.12100982666016 with parameters: {'lr': 2.782197177897237e-05}. Best is trial#21 with value: 74.12100982666016.
